# The Renormalization Group (RG) method #
##  for the false Duffing equation ##

Support material for the book __Perturbation Methods from a Backward Error Point of View__ by Robert M. Corless and Nicolas Fillion, in preparation.

This section of the book follows the method given in the very nice 2012 SIAM Review (Education) paper ["The Renormalization Group: A Perturbation Method for the Graduate Curriculum"](https://doi.org/10.1137/080731967) by Eleftherios Kirkinis. 


Copyright (c) 2024 Robert M. Corless

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.



## Preliminaries ##

First, some setup: a macro to allow easy typing of $\varepsilon$, and some helper utilities.

In [1]:
starting_time := time():
macro( ep = varepsilon );

kilobytes used=1676, alloc=5424, time=0.30

$$\varepsilon$$

In [2]:
simplify( residual );

$$\mathit{residual}$$

## Input parameters: the DE and the order we work to ##

This example is the false Duffing equation.  

In [3]:
y := 'y':
residual := 'residual':
WeaklyNonlinear := diff(y(t), t, t) + diff(y(t), t) + ep*y(t)^3

$$\frac{d^{2}}{d t^{2}}y \left(t \right)+\frac{d}{d t}y \left(t \right)+\varepsilon  y \left(t \right)^{3}$$

Choose the order to work to.  We will get the solution to $O(\varepsilon^{N+1})$, first with secularity, and then without.  

In [4]:
N := 1;  # Taking N=0 breaks some assumptions below, so choose N > 0.

                                    N := 1

$$1$$

## Compute the naive secular solution ##

In [5]:
ys := Array(0 .. N): # Placeholder for secular solution terms
rs := Array(0 .. N): # Placeholder for residuals

The initial approximation is a solution to $y'' + y' = 0$.

In [6]:
ys[0] := A + B*exp(-t);

$$A +B \,{\mathrm e}^{-t}$$

In [7]:
zs := ys[0]: # Current secular solution in the variable "zs"

$$A +B \,{\mathrm e}^{-t}$$

Build up the regular perturbation solution using our fast "particular solution" solver. We ignore initial conditions entirely; we will fix them up at the end if we need to.

Regular perturbation solves the linearized system $y'' + y =$ the negative of the coefficient of $\varepsilon^k$ in the residual of $y_{k-1}$.


In [8]:
loop_start := time():
times := Array(1 .. N):
for k to N do
    residuals[k - 1] := combine(expand(
                          coeff(eval(WeaklyNonlinear, y(t) = zs), ep, k)),
                          exp);
    newans := rhs( dsolve( 
      {diff(y(t),t,t) + diff(y(t),t) = - residuals[k-1],
       y(0)=0, D(y)(0)=0}, y(t)
                          )
                );
    ys[k] := newans;
    zs := zs + ys[k]*ep^k;
    times[k] := time();
end do:

kilobytes used=5705, alloc=9814, time=0.70

In [9]:
zs;

$$A +B \,{\mathrm e}^{-t}+\left(\frac{\left(-2 A^{3}+6 A^{2} B \left(t +1\right)+6 A \,B^{2}+B^{3}\right) {\mathrm e}^{-t}}{2}-\frac{B^{3} {\mathrm e}^{-3 t}}{6}-\frac{3 A \,B^{2} {\mathrm e}^{-2 t}}{2}+\left(-t +1\right) A^{3}-3 A^{2} B -\frac{3 A \,B^{2}}{2}-\frac{B^{3}}{3}\right) \varepsilon$$

Always compute the final residual.  Here, we are just computing the leading coefficient of the final residual; that is enough for us here.

In [10]:
residuals[N] := CodeTools:-Usage(
  map(LargeExpressions:-Veil[K], 
  series(leadterm(
    combine(expand(eval(WeaklyNonlinear, y(t) = zs)), exp)),
       ep)
     )                          );

memory used=2.36MiB, alloc change=0 bytes, cpu time=16.00ms, real time=18.00ms, gc time=0ns

$$-\frac{K_{1}}{2} \varepsilon^{2}$$

For this equation, the secular terms arise from A and from B in different ways.

In [11]:
as := eval(zs, exp=0 );

$$A +\left(\left(-t +1\right) A^{3}-3 A^{2} B -\frac{3 A \,B^{2}}{2}-\frac{B^{3}}{3}\right) \varepsilon$$

In [12]:
tmp := eval( zs, [exp(-t)=Z, exp(-2*t)=Z^2, exp(-3*t)=Z^3]);

$$A +B Z +\left(\frac{\left(-2 A^{3}+6 A^{2} B \left(t +1\right)+6 A \,B^{2}+B^{3}\right) Z}{2}-\frac{B^{3} Z^{3}}{6}-\frac{3 A \,B^{2} Z^{2}}{2}+\left(-t +1\right) A^{3}-3 A^{2} B -\frac{3 A \,B^{2}}{2}-\frac{B^{3}}{3}\right) \varepsilon$$

In [13]:
bs := coeff(tmp, Z );

$$B +\left(-A^{3}+3 A^{2} B \left(t +1\right)+3 A \,B^{2}+\frac{B^{3}}{2}\right) \varepsilon$$

In [14]:
eval( series( diff(as,t)/as, ep, N+1 ), t=0 );

$$-A^{2} \varepsilon +\mathrm{O}\left(\varepsilon^{2}\right)$$

In [15]:
eval( series( diff(bs,t)/bs, ep, N+1), t=0 );

$$3 A^{2} \varepsilon +\mathrm{O}\left(\varepsilon^{2}\right)$$

These are more complicated than those of the other class of equation.  At first order they are simple enough, but at second order they become a set of _rational_ coupled differential equations.  Luckily the first order is already enough to let us solve the problem quite adequately.